In [12]:
# imports 
import functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import ensemble, preprocessing, metrics

In [6]:
x_train, y_train = functions.get_data()

In [7]:
print(len(train_x))
print(len(train_y))
print(len(test_x))
print(len(test_y))

29138
29138
7285
7285


In [26]:
###### Random forest ######
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_x, train_y)

# predict
test_y_predicted = forest.predict(test_x)

/Users/ghost/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
###### Random forest -- Feature Importance ######

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")
for f in range(train_x.shape[1]):
    print("%d. %s (%f)" % (f + 1, train_x.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. Credit Score (0.258303)
2. Current Loan Amount (0.108093)
3. Annual Income (0.084643)
4. Monthly Debt (0.079641)
5. Maximum Open Credit (0.077889)
6. Current Credit Balance (0.076574)
7. Years of Credit History (0.074731)
8. Months since last delinquent (0.069601)
9. Number of Open Accounts (0.054757)
10. Years in current job (0.038399)
11. Number of Credit Problems (0.010240)
12. Term (0.010085)
13. Purpose_Debt Consolidation (0.007618)
14. Bankruptcies (0.006886)
15. Home Ownership_Rent (0.006008)
16. Home Ownership_Home Mortgage (0.005450)
17. Purpose_Home Improvements (0.004437)
18. Purpose_other (0.004387)
19. Home Ownership_Own Home (0.003855)
20. Tax Liens (0.003551)
21. Purpose_Other (0.003184)
22. Purpose_Business Loan (0.002760)
23. Purpose_small_business (0.001827)
24. Purpose_Medical Bills (0.001570)
25. Purpose_Buy a Car (0.001507)
26. Purpose_Take a Trip (0.000911)
27. Purpose_Buy House (0.000876)
28. Purpose_vacation (0.000734)
29. Purpose_major_purch

In [28]:
###### Random forest -- Evaluation ######

# accuracy
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print("Accuracy: {0}".format(accuracy))

# AUC index, excellent = 0.9~1, good = 0.8~0.9, soso = 0.7~0.8, not good = 0.6~0.7, bad = 0.6~0.7
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print("AUC: {0}".format(auc))

Accuracy: 0.9179135209334248
AUC: 0.8002215141867184
